# MEG Research

### _Project Description:_
Section III (Neural generation of electromagnetic fields) contains the basic physics of the signal generation in MEG. Subsections III.C and III.E.2 are the most relevant ones for the project. Implement equation 34 and compare the corresponding magnetic field distribution to the distribution that you get by calculating B with a numerical Biot-Savart implementation. The main outcome of the project is to assess whether the approximation of eq. 34 deviates significantly from the Biot-Savart solution in case of our "phantom head" that contains the manufactured current triangle loops with a 5 mm base length. 

In [1]:
import sympy as sp
import numpy as np
import math as math
import scipy.constants as const
import scipy.integrate as integrate
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd

## Sensor Locations & Normal Vector Data

In [2]:
meg_data = pd.read_csv('/Users/audrey/Desktop/meg_data.csv', comment = "#", header=None, skipinitialspace = False)
meg_data

,0,1,2,3,4,5
0,-0.1066,0.0464,-0.0604,-0.982327,0.186741,0.013541
1,-0.1020,0.0631,-0.0256,-0.959313,0.282039,-0.012738
2,-0.1085,0.0302,-0.0266,-0.995417,0.095198,-0.002369
3,-0.1099,0.0131,-0.0627,-0.999748,0.021179,0.012336
4,-0.1074,0.0329,0.0080,-0.993444,0.101232,0.051711
...,...,...,...,...,...,...
97,0.0488,-0.0994,-0.0621,0.429621,-0.903014,0.011048
98,0.1087,-0.0033,-0.0280,0.996532,-0.080951,-0.020708
99,0.1068,-0.0206,-0.0621,0.976343,-0.215775,0.011820
100,0.0951,-0.0524,-0.0620,0.879009,-0.476686,0.011789


In [3]:
# Sensor Location Coordinates
x_sensorlocation = meg_data[0]
y_sensorlocation = meg_data[1]
z_sensorlocation = meg_data[2]

# Normal Vector Coordinates
x_normalvector = meg_data[3]
y_normalvector = meg_data[4]
z_normalvector = meg_data[5]


## Calculating the Magnetic Field (Assumption Method)

Here, "I" is the current from the sink at $r_{1}$ to the source at $r_{2}$, "Q" referrs to the current dipole. Q is defined as:

$$
Q = I\, \times\, (r_{2} - r_{1})
$$


### Magnetic Field Determined Via Assumption Method
Using the assumption method presented by Ilmoniemi _et al._ (1985) and by Sarvas (1987), the magnetic field in rectangular coordinates can be calculated by:

$$
\mathbf{B(r)} = \frac{\mu_{0}}{4 \pi} \frac{F\mathbf{Q} \times \mathbf{r_{Q}} - (\mathbf{Q} \times \mathbf{r_{Q}} \cdot \mathbf{r}) \nabla F(\mathbf{r},\mathbf{r_{Q}})}{F(\mathbf{r},\mathbf{r_{Q}})^{2}}
$$

where
$$
F(\mathbf{r}, \mathbf{r_{Q}}) = a(ra + r^{2} - \mathbf{r_{Q}} \cdot \mathbf{r})
$$

and
$$
\nabla F(\mathbf{r}, \mathbf{r_{Q}}) = (r^{-1}a^{2} + a^{-1}\mathbf{a} \cdot \mathbf{r} + 2a +2r)\mathbf{r} - (a + 2r + a^{-1}\mathbf{a} \cdot \mathbf{r})\mathbf{r_{Q}}
$$

with $a=(r-r_{Q})$, $\mathbf{a} = |a|$, and $r=|\mathbf{r}|$.

In [4]:
# Vacuum magnetic permeability: "permeability of free space"
from astropy.constants import mu0

In [5]:
def F(r_vector, rQ_vector):
    a_vector = r_vector - rQ_vector
    a = np.linalg.norm(a_vector)
    r = np.linalg.norm(r_vector)
    F = a * ((r * a) + (r**2) - np.dot(r_vector, rQ_vector))
    return F

def delF(r_vector, rQ_vector):
    a_vector = r_vector - rQ_vector
    a = np.linalg.norm(a_vector)
    r = np.linalg.norm(r_vector)
    delF = (((a**2 / r) + (np.dot(a_vecor, r_vector) / a) + (2 * a) + (2 * r)) * r_vector) - ((a + (2 * r) + (np.dot(a_vector, r_vector)/a)) * rQ_vector)
    return delF

In [6]:
# Calculating the Magnetic Field

def B(r_vector, rQ_vector, Q):
    B_numerator = (F(r_vector, rQ_vector) * np.cross(Q, rQ_vector)) - ((np.dot(np.cross(Q, rQ_vector), r_vector))*delF(r_vector, rQ_vector))
    B_denominator = F(r_vector, rQ_vector)**2
    magnetic_field = (mu0 / (4 * np.pi)) * (B_numerator / B_denominator)
    return magnetic_field

## Calculating the Magnetic Field (Biot-Savart Law)

### Magnetic Field From Biot-Savart Law
The magnetic field of a steady line current is geven by the **Biot-Savart law**:

$$
\mathbf{B(r)} = \frac{\mu_{0}}{4\pi} \int \frac{\mathbf{I}  \times \hat{\mathcal{R}}}{\mathscr{R}^{2}} dl' = \frac{\mu_{0}}{4\pi} I \int \frac{d\mathbf{l}' \times \hat{\mathcal{R}}}{\mathscr{R} ^{2}}
$$

Note:
- Current = **I** (with direction)
- Current Magnitude = |**I**| $\rightarrow$ labeled as "I"
- Distance = $d\mathbf{l}'$

### Position Vector
Position Vector:
$$
\mathbf{r} = x\, \hat{\mathbf{x}} + y\, \hat{\mathbf{y}} + z\, \hat{\mathbf{z}}
$$

Position Vector Magnitude:
$$
r = \sqrt{x^{2} + y^{2} + z^{2}}
$$

Distance from the origin:
$$
\hat{\mathbf{r}} = \frac{\mathbf{r}}{r} = \frac{x\, \hat{\mathbf{x}} + y\, \hat{\mathbf{y}} + z\, \hat{\mathbf{z}}}{\sqrt{x^{2} + y^{2} + z^{2}}}
$$

### Displacement Vector

Infinitesimal Displacement Vector:
$$
d\mathbf{l} = dx\, \hat{\mathbf{x}} + dy\, \hat{\mathbf{y}} + dz\, \hat{\mathbf{z}}
$$

Infinitesimal Displacement Vector Magnitude:
$$
dl = \sqrt{dx\,^{2} + dy\,^{2} + dz\,^{2}}
$$


### Separation Vector

Separation Vector
$$
\mathbf{\mathcal{R}} = \mathbf{r} - \mathbf{r}'
$$

Separation Vector Magnitude
$$
\mathscr{R} = |\mathbf{r} - \mathbf{r}'|
$$

Unit Vector in Direction From $\mathbf{r}'$ to $\mathbf{r}$
$$
\hat{\mathcal{R}} = \frac{\mathcal{R}}{\mathscr{R}} = \frac{\mathbf{r} - \mathbf{r}'}{|\mathbf{r} - \mathbf{r}'|}
$$

#### In Cartesian Coordinates:
$$
\mathcal{R} = (x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}
$$

$$
\mathscr{R} = \sqrt{(x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}}
$$

$$
\hat{\mathcal{R}} = \frac{(x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}}{\sqrt{(x-x')\, \hat{\mathbf{x}} + (y-y')\, \hat{\mathbf{y}} + (z-z')\, \hat{\mathbf{z}}}}
$$

Definitions:
- Source Point: $\mathbf{r}'$
- Field Point: $\mathbf{r}$
- Separation Vector: $\mathbf{\mathcal{R}}$
- Separation Vector Magnitude: $\mathscr{R}$

In [7]:
# Here, r and r_prime are 3D vectors with x, y, and z components corresponding to r, r_prime[0, 1, 2]

def scriptR_vector(r, r_prime):
    sep_vector = (r[0]-r[0]) + (r[1]-r_prime[1]) + (r[2]-r_prime[2])
    sep_vector_mag = np.sqrt(sep_vector)
    return sep_vector/sep_vector_mag

def scriptR_magnitude(r, r_prime):
    magnitude = np.sqrt((r[0] - r_prime[0]) + (r[1] - r_prime[1]) + (r[2] - r_prime[2]))
    return magnitude

# I, r, r_prime, dl_prime are 3D vectors with x, y, and z components

def B_current_vector(I, r, r_prime, dl_prime):
    scriptR = scriptR_vector(r, r_prime)
    scriptR_mag = scriptR_magnitude(r, r_prime)
    int_function = np.cross(I, scriptR) / scriptR_mag^2
    B = (mu0/(4*np.pi)) * sp.integrate(int_function, dl_prime)
    #evaluate integral
    #B.subs().n()
    return B

def B_length_vector(I, r, r_prime, dl_prime):
    scriptR = scriptR_vector(r, r_prime)
    scriptR_mag = scriptR_magnitude(r, r_prime)
    int_function = np.cross(dl_prime, scriptR) / scriptR_mag^2
    I_mag = np.sqrt(I[0]^2 + I[1]^2 + I[2]^2)
    B = (mu0/(4*np.pi)) * I_mag * sp.integrate(int_function)
    return B


## Defining Vector Projection Function

#### Vector Projection:
The projection of $\mathbf{a}$ onto $\mathbf{b}$ is:
$$
\mathbf{a} = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{\, b\, }||^{2}} \, \mathbf{b}
$$

In [8]:
def vectorProjection(a_x, a_y, a_z, b_x, b_y, b_z):
    x_dot = np.dot(a_x, b_x)
    y_dot = np.dot(a_y, b_y)
    z_dot = np.dot(a_z, b_z)
    total_dot = x_dot + y_dot + z_dot
    
    b_vector = np.array([b_x, b_y, b_z])
    b_magnitude = np.linalg.norm(b_vector)
    
    vector_projection = total_dot * b_vector / (b_magnitude**2)
    
    return vector_projection

In [9]:
projection_component_array = vectorProjection(x_sensorlocation, y_sensorlocation, z_sensorlocation, x_normalvector, y_normalvector, z_normalvector)


In [10]:
# Projection Components
projection_xcomp = projection_component_array[0]
projection_ycomp = projection_component_array[1]
projection_zcomp = projection_component_array[2]

In [11]:
def sensor_projection_vec(x, y, z):
    for i, j, k in zip(x, y, z):
        sensor_vec = np.array([i, j, k])
        print(sensor_vec)

In [12]:
projections = sensor_projection_vec(projection_xcomp, projection_ycomp, projection_zcomp)


[-0.11219877  0.02132906  0.00154662]
[-0.10957018  0.03221374 -0.0014549 ]
[-0.11369388  0.01087326 -0.00027058]
[-0.11418855  0.00241901  0.00140899]
[-0.11346853  0.01156245  0.00590629]
[-0.10568279  0.01717897  0.03978841]
[-0.10681434 -0.00498582  0.04013883]
[-0.11354665 -0.00964977  0.0077485 ]
[-0.09988957  0.05446385  0.01003   ]
[-0.09817151  0.04021512  0.04232038]
[-0.08061597  0.03556511  0.07268392]
[-0.10910885  0.03281304  0.00800983]
[-0.08618326  0.01556188  0.07333438]
[-0.05494448  0.01699965  0.09868035]
[-0.05614296 -0.00315057  0.09940506]
[-0.08692225 -0.00667418  0.0738005 ]
[-0.07092634  0.08767323  0.01807409]
[-0.03311492  0.10674581  0.02356692]
[-0.03053589  0.09634198  0.05321466]
[-0.07801672  0.07081007  0.04408013]
[-0.0339985   0.06921114  0.08425573]
[3.90623292e-05 4.53899697e-02 1.04817250e-01]
[-0.01600962  0.02044924  0.11122073]
[-0.0352485   0.04728107  0.0978123 ]
[-0.01602892 -0.00150893  0.11307311]
[0.01708234 0.00458868 0.11283771]
[ 0.01